In [39]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

In [40]:
def create_line_graph_popularity(name, country, date, player_search, country_search):
    data = pd.DataFrame({
        'Date': pd.to_datetime(date),
        f"{name}'s search": player_search,
        f"{country}'s search": country_search
    })

    # Melt DataFrame for interactive plotting
    data_melted = data.melt(id_vars=['Date'], var_name='Search Type', value_name='Search Volume')

    # Calculate correlation
    correlation = np.corrcoef(player_search, country_search)[0, 1]

    # Create interactive plot
    fig = px.line(
        data_melted, x='Date', y='Search Volume', color='Search Type',
        title=f"{name}'s Popularity Graph",
        labels={'Search Volume': 'Search Volume', 'Date': 'Date'},
        template="plotly_white" 
    )

    fig.add_annotation(
        x=data['Date'].iloc[len(data) // 2], 
        y=max(player_search) * 0.9,
        text=f"Correlation: {correlation:.2f}",
        showarrow=False,
        font=dict(size=14, color="blue"),
        bgcolor="white"
    )
    if name == "Magnus Carlsen":
        fig.write_html("../docs/plots/magnus_gtrends_plot.html", full_html=False)
        print("found him") # saving plot wanted as html file for website usasge 
    # Show interactive plot
    fig.show()

In [42]:
import sqlite3
def player_info_returner(name,country_code):
    conn = sqlite3.connect("../data/chess.db")
    cursor = conn.cursor()


    country = cursor.execute(f"SELECT date, search_rate FROM country_gtrends WHERE country = '{country_code}'").fetchall()
    player = cursor.execute(f"SELECT date, search_rate FROM players_gtrends WHERE name='{name}'").fetchall()

    dates = [date[0] for date in country]
    dates = pd.to_datetime(dates, format="%b %Y")
    country_search=[rate[1] for rate in country]
    player_search=[rate[1] for rate in player[:-1]]

    conn.close()
    return dates,country_search,player_search

In [43]:
top_players =[
    {'name': 'Magnus_Carlsen','country':'NO','country_name':'Norway'},
    {'name': 'Fabiano_Caruana','country':'US','country_name':'United States of America'},
    {'name': 'Hikaru_Nakamura','country':'US','country_name':'United States of America'},
    {'name': 'Arjun_Erigaisi','country':'IN','country_name':'India'},
    {'name': 'Gukesh_Dommaraju','country':'IN','country_name':'India'},
    {'name': 'Nodirbek_Abdusattorov','country':'UZ','country_name':'Uzbekistan'},
    {'name': 'Alireza_Firouzja','country':'FR','country_name':'France'},
    {'name': 'Ian_Nepomniachtchi','country':'RU','country_name':'Russia'},
    {'name': 'Yi_Wei','country':'CN','country_name':'China'},
    {'name': 'Viswanathan_Anand','country':'IN','country_name':'India'}
]
for players in top_players:
    date,country_search,player_search = player_info_returner(players['name'].replace('_', ' '),players['country'])
    create_line_graph_popularity(players["name"].replace('_', ' '),players["country_name"],date,player_search,country_search)

found him
